In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv
from collections import OrderedDict
import requests
import json
import numpy as np
load_dotenv()
API_KEY = os.getenv('SECRET_KEY')

In [2]:
#takes dictonary and returns list of opening prices
def get_open_price(d):
    opening_prices = []
    for d_k in d.keys():
        if d[d_k]['Open'] != None:
            opening_prices.append(d[d_k]['Open'])
    return opening_prices

In [3]:
#1.Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
try:
    r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2016-12-31&end_date=2017-12-31&api_key={}' .format(API_KEY))
    assert r.status_code == 200
except:
    print('Retrieve Failed. Status Code: {}' .format(r.status_code))

In [4]:
#2.Convert the returned JSON object into a Python dictionary.
r_json = r.json()
keys = r_json['dataset_data']['column_names']
afx_x_date_dict = OrderedDict()
for i in r_json['dataset_data']['data']:
    date_key = i[:1]
    key = keys[1:]
    data = i[1:]
    day_dict = dict(zip(key, data))
    afx_x_date_dict.update({date_key[0]:day_dict})

In [5]:
#3.Calculate what the highest and lowest opening prices were for the stock in this period.
opening_prices = get_open_price(afx_x_date_dict)
print('Max Price: {}' .format(np.max(opening_prices)))
print('Min Price: {}' .format(np.min(opening_prices)))

Max Price: 53.11
Min Price: 34.0


In [6]:
keys

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [7]:
#4.What was the largest change in any one day (based on High and Low price)?
delta = []
for k in afx_x_date_dict.keys():
    high = afx_x_date_dict[k]['High']
    low = afx_x_date_dict[k]['Low']
    delta.append(high - low)
print('Largest One Day Change: {:.2f}' .format(np.max(delta)))

Largest One Day Change: 2.81


In [8]:
#5.What was the largest change between any two days (based on Closing Price)? Consecutive???
closing_price = []
closing_diff = []
for k in afx_x_date_dict.keys():
    closing_price.append(afx_x_date_dict[k]['Close'])
for i in range(0, len(closing_price)):
    try:
        closing_diff.append(np.diff(closing_price[i:i+2])[0])
    except:
        pass
print('Largest Change Between 2 Consecutive Days: {:.2f}' .format(closing_diff[np.argmax(np.abs(closing_diff))]))

Largest Change Between 2 Consecutive Days: 2.56


In [9]:
#6.What was the average daily trading volume during this year?
daily_volume = []
for k in afx_x_date_dict.keys():
    daily_volume.append(afx_x_date_dict[k]['Traded Volume'])
print('Average Daily Trading Volume: {:.2f}' .format(np.mean(daily_volume)))

Average Daily Trading Volume: 89124.34


In [10]:
#7.What was the median trading volume during this year.
sorted_daily = np.sort(daily_volume)
print('Median Trading Volume: {}' .format(sorted_daily[int(len(sorted_daily)*.5): int(len(sorted_daily)*.5)+1][0]))

Median Trading Volume: 76286.0
